#### First get the candidate model parts (Head and Feature Extractor)

In [5]:
#Only model weights in GPU: 7.9 GB
#Batch of 4 images pairs forward pass: +13.5GB
#Batch of 4 images pairs backward pass: +5GB
#Fordward pass time GPU batch 4 pairs: 300 ms
#forward + Backward  pass time CPU batch 4 pairs: 13 segs + 40 segs

In [1]:
import os
import torch
import torch.nn as nn
import time
from torch.distributed.rpc import init_rpc
from drecg.feature_extraction.distributed import define_model_for_tune


os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'
init_rpc('worker', rank=0, world_size=1)


devices_list = [torch.device("cuda:0"), torch.device("cpu")]
model_pipe = define_model_for_tune(devices_list, microbatch_num=2)

loss_fn = nn.BCEWithLogitsLoss()
y_true = torch.ones(32, 1, dtype=torch.float32).to(devices_list[-1])

model_pipe.train()
adam_w = torch.optim.AdamW(model_pipe.parameters(), lr=1e-3)



config.json not found in HuggingFace Hub.


In [ ]:
for i in range(2):
    init_time = time.time()
    dummy_img_a = torch.rand(32, 3, 224, 224).to(devices_list[0])
    dummy_img_b = torch.rand(32, 3, 224, 224).to(devices_list[0])
    adam_w.zero_grad()
    out = model_pipe((dummy_img_a, dummy_img_b))
    out = out.local_value()
    loss = loss_fn(out, y_true)
    loss.backward()
    adam_w.step()
    total_time_segs = time.time() - init_time
    print("iter: {}, loss: {}, time: {} segs".format(i, loss.item(), total_time_segs))

In [1]:
from drecg.data.utils import create_dataloader_validation
from drecg.feature_extraction.utils import VitLaionPreProcess

transforms = VitLaionPreProcess()
validation_dataloader = create_dataloader_validation(transforms=transforms, batch_size=16)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
2023-04-26 19:46:01.186718: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-26 19:46:01.341945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 19:46:01.859075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
next(iter(validation_dataloader))

[[tensor([[[[-1.5879, -1.5879, -1.5879,  ..., -1.5879, -1.5879, -1.5879],
            [-1.5879, -1.5879, -1.5879,  ..., -1.5879, -1.5879, -1.5879],
            [-1.5879, -1.5879, -1.5879,  ..., -1.5879, -1.5879, -1.5879],
            ...,
            [-1.4419, -1.4711, -1.5003,  ..., -1.5587, -1.5295, -1.5441],
            [-1.4273, -1.4419, -1.4565,  ..., -1.5587, -1.5295, -1.5295],
            [-1.3981, -1.4127, -1.4127,  ..., -1.5587, -1.5295, -1.5295]],
  
           [[-1.5420, -1.5420, -1.5420,  ..., -1.5420, -1.5420, -1.5420],
            [-1.5420, -1.5420, -1.5420,  ..., -1.5420, -1.5420, -1.5420],
            [-1.5420, -1.5420, -1.5420,  ..., -1.5420, -1.5420, -1.5420],
            ...,
            [-1.3919, -1.4219, -1.4519,  ..., -1.5120, -1.4820, -1.4970],
            [-1.3769, -1.3919, -1.4069,  ..., -1.5120, -1.4820, -1.4820],
            [-1.3469, -1.3619, -1.3619,  ..., -1.5120, -1.4820, -1.4820]],
  
           [[-1.2811, -1.2811, -1.2811,  ..., -1.2811, -1.2811, -1.281

In [ ]:
folder = '/home/daniel/data_dogs/validation'
## Zip folder
!zip -r validation.zip ./validation/

#transfer by scp to remote server using ssh config name may_serve
!scp validation.zip may_serve:/home/